# Content Based Movie Recommendation

In [25]:
import numpy as np
import pandas as pd
import re
pd.set_option('display.max_columns', None)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from scipy.sparse import csr_matrix
from numba import jit

### Import Dataset

In [26]:
# Import movie dataset (combined metadata), subset the data to 10k rows for computational conviencies
whole_df = pd.read_csv('combined_metadata_table.csv')

### Pick A Movie (Fake Search Engine)

In [27]:
def identify_movie(your_pick, whole_df):
    return whole_df[whole_df['title'].str.contains(your_pick, flags=re.IGNORECASE, regex=True)]

In [28]:
your_pick = 'Jumanji'
identify_movie(your_pick, whole_df)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,metascore,reviews_from_users,reviews_from_critics,id,overview,popularity,revenue,tagline
15078,tt0113497,Jumanji,Jumanji,1995,2/16/96,"Adventure, Comedy, Family",104,USA,"English, French",Joe Johnston,"Jonathan Hensleigh, Greg Taylor",TriStar Pictures,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",When two kids find and play a magical board ga...,7.0,278445,39.0,220.0,101.0,8844,When siblings Judy and Peter discover an encha...,17.015539,262797249.0,Roll the dice and unleash the excitement!


### Subset Dataset

In [29]:
def subset_by_genre(your_pick, whole_df):
    genres = whole_df[whole_df['title'] == your_pick]['genre']
    genres = list(genres)[0].split(', ')
    masks = []
    for genre in genres:
        mask = whole_df['genre'].str.contains(genre)
        masks.append(mask)
    mask = masks[0]
    for i in range(1,len(masks)):
        mask = mask | masks[i]
    return whole_df[mask]

In [30]:
def subset_by_year(your_pick, whole_df, year_range=30):
    year = whole_df[whole_df['title'] == your_pick]['year']
    year = list(year)[0]
    window = [year-year_range, year+year_range]
    mask = (whole_df['year'] >= window[0]) & (whole_df['year'] <= window[1])
    return whole_df[mask]

In [31]:
def subset_by_language(your_pick, whole_df):
    languages = whole_df[whole_df['title'] == your_pick]['language']
    languages = list(languages)[0].split(', ')
    masks = []
    for language in languages:
        mask = whole_df['language'].str.contains(language)
        masks.append(mask)
    mask = masks[0]
    for i in range(1,len(masks)):
        mask = mask | masks[i]
    return whole_df[mask]

In [32]:
subset_df = subset_by_genre(your_pick, whole_df)
source = subset_by_year(your_pick, subset_df)
#source = subset_by_language(your_pick, source)

In [33]:
print('Subset Size:', source.shape[0])
source.head(3)

Subset Size: 12735


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,writer,production_company,actors,description,avg_vote,votes,metascore,reviews_from_users,reviews_from_critics,id,overview,popularity,revenue,tagline
5889,tt0059126,Dr. Who and the Daleks,Dr. Who and the Daleks,1965,8/23/65,"Adventure, Family, Sci-Fi",82,UK,English,Gordon Flemyng,"Terry Nation, Milton Subotsky",AARU Productions,"Peter Cushing, Roy Castle, Jennie Linden, Robe...",An eccentric inventor and his companions trave...,5.8,3497,NaN,63.0,36.0,26581,Scientist Doctor Who accidentally activates hi...,2.187942,0.0,Now on the Big Screen in COLOUR!
5891,tt0059269,A High Wind in Jamaica,A High Wind in Jamaica,1965,6/4/65,"Adventure, Drama",103,UK,"English, Spanish",Alexander Mackendrick,"Stanley Mann, Ronald Harwood",Twentieth Century-Fox Productions,"Anthony Quinn, James Coburn, Dennis Price, Lil...","In 1870, a Jamaican colonial family sends its ...",6.6,1716,NaN,31.0,18.0,38340,"In 1870, a Jamaican colonial family sends its ...",1.235903,0.0,NaN
5892,tt0059014,Carry On Cowboy,Carry On Cowboy,1965,5/27/66,"Comedy, Western",93,UK,English,Gerald Thomas,Talbot Rothwell,Peter Rogers Productions,"Sidney James, Kenneth Williams, Jim Dale, Char...",Stodge City is in the grip of the Rumpo Kid an...,6.2,2463,NaN,24.0,8.0,5065,Stodge City is in the grip of the Rumpo Kid an...,5.236409,0.0,How the west was lost!


### Data Processing

In [34]:
# Delete spaces in names to make names more unique
# eg. 'firstname lastname' -> 'firstnamelastname'
column_with_names = ['director', 'writer', 'production_company', 'actors']
source = source.copy()
for col in column_with_names:
    source[str(col)] = source[str(col)].str.replace(' ', '', regex=True)
    source[str(col)] = source[str(col)].str.replace(',', ' ', regex=True)

### Compute TFIDF

In [35]:
# Suggestion: remove the spaces for the names, to solving clustering ambiguity
# Issue: how to deal with numbers ???
columns = ['country', 'director','writer', 
           'production_company', 'actors',
           'description','overview', 'tagline']

In [36]:
# Converts the source dataframe into single string for tfidf computation
df = {}
movies = []
titles = []
imdbid = []
for i in range(source.shape[0]):
    row = source.iloc[i]
    row_str = ''
    titles.append(row['title'])
    imdbid.append(row['imdb_title_id'])
    for column in columns:
        row_str += str(row[column])
        row_str += ' '
    movies.append(row_str)

df['IMDBid'] = imdbid
df['Title'] = titles
df['Content'] = movies
df = pd.DataFrame(df)

In [37]:
df.head()

,IMDBid,Title,Content
0,tt0059126,Dr. Who and the Daleks,UK GordonFlemyng TerryNation MiltonSubotsky AA...
1,tt0059269,A High Wind in Jamaica,UK AlexanderMackendrick StanleyMann RonaldHarw...
2,tt0059014,Carry On Cowboy,UK GeraldThomas TalbotRothwell PeterRogersProd...
3,tt0059800,Thunderball,UK TerenceYoung RichardMaibaum JohnHopkins Eon...
4,tt0059362,The Knack ...and How to Get It,UK RichardLester CharlesWood AnnJellicoe Woodf...


In [39]:
def to_tfidf():
    V = TfidfVectorizer()
    X = V.fit_transform(df['Content'])
    
    print('X shape:', X.shape)
    return V, X

V, X = to_tfidf()

X shape: (12735, 159277)


### Compute Similarity

In [24]:
'''
# Dimensionality reduction
svd = TruncatedSVD(n_components=10)
X = svd.fit_transform(X)
print('X shape after SVD:', X.shape)


# Compute similarity of movie for SVD
index = df[df['Title'] == your_pick].index[0]
d1 = X[index]
mag_d1 = np.linalg.norm(d1)
dist = []
for i in range(X.shape[0]):
    row = X[i]
    dot_product_xy = np.dot(d1, row)
    mag_row = np.linalg.norm(row)
    x_time_y = mag_d1 * mag_row
    dist.append(dot_product_xy/x_time_y) 
dist_series = pd.Series(dist)
dist_series = dist_series.sort_values(ascending=False)
dist_series.iloc[1:6]
dist_series = pd.DataFrame(dist_series)
'''

"\n# Dimensionality reduction\nsvd = TruncatedSVD(n_components=10)\nX = svd.fit_transform(X)\nprint('X shape after SVD:', X.shape)\n\n\n# Compute similarity of movie for SVD\nindex = df[df['Title'] == your_pick].index[0]\nd1 = X[index]\nmag_d1 = np.linalg.norm(d1)\ndist = []\nfor i in range(X.shape[0]):\n    row = X[i]\n    dot_product_xy = np.dot(d1, row)\n    mag_row = np.linalg.norm(row)\n    x_time_y = mag_d1 * mag_row\n    dist.append(dot_product_xy/x_time_y) \ndist_series = pd.Series(dist)\ndist_series = dist_series.sort_values(ascending=False)\ndist_series.iloc[1:6]\ndist_series = pd.DataFrame(dist_series)\n"

In [42]:
'''
# Compute similarity of movie: Melvin and Howard
index = df[df['Title'] == your_pick].index[0]
d1 = list(csr_matrix.toarray(X[index]))
mag_d1 = np.linalg.norm(d1)
dist = []
for i in range(X.shape[0]):
    row = list(csr_matrix.toarray(X[i]))
    dot_product_xy = np.multiply(d1, row).sum(1)
    mag_row = np.linalg.norm(row)
    x_time_y = mag_d1 * mag_row
    dist.append(dot_product_xy/x_time_y) 
dist_series = pd.Series(dist)
dist_series = dist_series.sort_values(ascending=False)
dist_series.iloc[1:6]
dist_series = pd.DataFrame(dist_series)
'''

scipy.sparse.csr.csr_matrix

In [43]:
# Compute similarity of movie: Melvin and Howard
@jit(nopython=True)
def to_cossim():
    index = df[df['Title'] == your_pick].index[0]
    d1 = np.array(list(csr_matrix.toarray(X[index])))
    mag_d1 = np.linalg.norm(d1)
    dist = []
    for i in range(X.shape[0]):
        row = np.array(list(csr_matrix.toarray(X[i])))
        dot_product_xy = np.multiply(d1, row).sum(1)
        mag_row = np.linalg.norm(row)
        x_time_y = mag_d1 * mag_row
        dist.append(dot_product_xy/x_time_y) 
    dist_series = pd.Series(dist)
    dist_series = dist_series.sort_values(ascending=False)
    dist_series.iloc[1:6]
    dist_series = pd.DataFrame(dist_series)

### Recommend Top 5 Movies

In [17]:
# Merge similarity scores with original dataframe to visualize
result = pd.merge(dist_series, df, how='inner', left_index=True, right_index=True)
result = result.rename({0: 'Cosine Similarity Score'}, axis='columns')

In [18]:
# Show closest results
result.head(6)

,Cosine Similarity Score,IMDBid,Title,Content
2965,[1.0000000000000002],tt0085995,National Lampoon's Vacation,USA HaroldRamis JohnHughes JohnHughes WarnerBr...
4023,[0.23926701067907094],tt0097958,National Lampoon's Christmas Vacation,USA JeremiahS.Chechik JohnHughes WarnerBros. C...
5548,[0.1860146536597548],tt0120434,Vegas Vacation,USA StephenKessler ElisaBell BobDucsay WarnerB...
3307,[0.15279257267950555],tt0089670,National Lampoon's European Vacation,USA AmyHeckerling JohnHughes RobertKlane Warne...
4660,[0.14477408723062454],tt0107290,Jurassic Park,USA StevenSpielberg MichaelCrichton MichaelCri...
4180,[0.1159607089009845],tt0099785,Home Alone,USA ChrisColumbus JohnHughes HughesEntertainme...


### Some Ideas for Future

In [19]:
# make a window of year range
# take the difference between year to see the closest
# make it fast: compute the tfidf once per day, put in cache
# Use PCA to speed up cosine similarity computation